In [ ]:
# Connect to Database
import os

database_url = os.environ["database_url2"]
verbose = True

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

session = sessionmaker(bind=create_engine(database_url, echo=False), autoflush=False)()


In [ ]:
# Select Daten aus der stunden Datenbank
from sqlalchemy import func

ereignisse = ["K", "COR"]
zeitpunkt = "Monat Oktober 2021"

from kostunden.database.models import Mitarbeiterereignis, Mitarbeiter, Kostenstelle

q = session.query(Kostenstelle.kennung, func.count(Kostenstelle.kennung)).select_from(Mitarbeiterereignis).join(Mitarbeiter).join(Kostenstelle, Kostenstelle.id == Mitarbeiter.kostenstelle_id) \
    .filter(Mitarbeiterereignis.datum >= '2021-10-01', Mitarbeiterereignis.datum <= '2021-10-31')\
    .filter(Mitarbeiterereignis.ganztaegig.in_(ereignisse))\
    .order_by(Kostenstelle.kennung)\
    .group_by(Kostenstelle.kennung)
  
kostenstelle_ausfalltage = {d[0]: d[1] for d in list(q)}

if verbose:
    print(kostenstelle_ausfalltage)

In [ ]:
# Select Daten aus der stunden Datenbank
from sqlalchemy import func

from kostunden.database.models import Mitarbeiterstunden

q = session.query(Kostenstelle.kennung, func.count(Kostenstelle.kennung)).select_from(Mitarbeiterstunden).join(Kostenstelle) \
    .filter(Mitarbeiterstunden.datum >= '2021-10-01', Mitarbeiterstunden.datum <= '2021-10-31')\
    .order_by(Kostenstelle.kennung) \
    .group_by(Kostenstelle.kennung)

kostenstelle_ausfuehrtage = {d[0]: d[1] for d in list(q)}

if verbose:
    print(kostenstelle_ausfuehrtage)
    

In [ ]:
# merge Listen Ausfalltage und Ausfühtage

q = session.query(Kostenstelle.kennung).order_by("kennung").all()
alle_kostenstellen = [k[0] for k in list(q)]

data = list()
for kostenstelle in alle_kostenstellen:
    # data.append({"kostenstelle": kostenstelle, "ausfuehrtage": kostenstelle_ausfuehrtage.get(kostenstelle, 0), "ausfalltage": kostenstelle_ausfalltage.get(kostenstelle, 0)})
    data.append({"Kostenstelle": kostenstelle, "Typ": "Ausführtage", "Anzahl": kostenstelle_ausfuehrtage.get(kostenstelle, 0)})
    data.append({"Kostenstelle": kostenstelle, "Typ": "Ausfalltage", "Anzahl": kostenstelle_ausfalltage.get(kostenstelle, 0)})

if verbose:
    print(data)

    

In [ ]:
# DataFrame erstellen

import pandas as pd

data_frame = pd.DataFrame(data)

if verbose:
    print(data_frame)


In [ ]:
import numpy as np

def show_values(axs, orient="v", space=.01):
    def _single(ax):
        if orient == "v":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height() + (p.get_height()*0.01)
                value = '{:.1f}'.format(p.get_height())
                ax.text(_x, _y, value, ha="center") 
        elif orient == "h":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() + float(space)
                _y = p.get_y() + p.get_height() - (p.get_height()*0.5)
                value = '{:.1f}'.format(p.get_width())
                ax.text(_x, _y, value, ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _single(ax)
    else:
        _single(axs)

In [ ]:
# Diagramm erstellen

import matplotlib.pyplot as plt
import seaborn as sn

sn.set(font_scale=1.0)
fig, ax = plt.subplots(figsize=(20,7))

palette = ['tab:green','tab:red']

p = sn.barplot(data = data_frame
            ,x = 'Kostenstelle'
            ,y = 'Anzahl'
            ,hue = 'Typ' 
            ,palette=palette
            )

ax.set_title(f"Ausfalltage im {zeitpunkt} wegen Krankheit ({', '.join(ereignisse)})", y=1, fontsize = 16)
show_values(p)
